# Assignment 2 – Markov Decision Processes

The primary description of this coursework is available on course page page. This is the Jupyter notebook you must complete and submit to receive marks. 

**You must follow all instructions given in this notebook.**

Restart the kernel and run all cells before submitting the notebook. This will guarantee that we will be able to run your code for testing.

Remember to save your work regularly. 

## Assignment Details
This assignment has two parts, both are contained within this one notebook. You are welcome to write code separately (e.g. in an IDE like Pycharm) which you then copy into the notebook to submit, but you must make sure the notebook runs without any additional resources.

In part one you will write code to solve a simple dice game using value iteration, the details of which are explained below. This part is worth 80% of the assignment grade.

In part two you will write about your implementation, explain any choices you made, and discuss the possible impact of changing the game mechanics. This part is worth 20% of the assignment grade.

## A Dice Game
The goal of the assignment is to solve the optimal strategy for a simple single-player points-based dice game using value iteration. The rules of the basic version of the game are as follows:
* You start with 0 points
* Roll three fair six-sided dice
* Now choose:
 * Stick, accept the values shown. If two or more dice show the same values, then all of them are flipped upside down: 1 becomes 6, 2 becomes 5, 3 becomes 4, and vice versa. The total is then added to your points and this is your final score.
 * OR reroll the dice. You may choose to hold any combination of the dice on the current value shown. Rerolling costs you 1 point – so during the game and perhaps even at the end your score may be negative. You then make this same choice again.
 
The best possible score for this game is 18 and is achieved by rolling three 1s on the first roll.

The reroll penalty prevents you from rolling forever to get this score. If the value of the current dice is greater than the expected value of rerolling them (accounting for the penalty), then you should stick.

The optimal decision is independent of your current score. It does not matter whether it is your first roll with a current score of 0, or your twentieth roll with a current score of -19 (in which case a positive end score is impossible), in either of these cases if you roll three 6s (which, if you stick, will only add 3 points) then you still expect to get a *better* end score by rerolling and taking the penalty. Almost any other roll will beat it, so it's still the right choice to maximise your score.

It is pretty obvious that you should stick on three 1s, and reroll on three 6s. Should you hold any of the 6s when you reroll? What about other values? What should you do if the dice come up 3, 4, 5?

We do not know what numbers will come up when we roll, but we do know exactly what the probability of any given roll is. This is the point of the probabilistic reasoning section of the unit; if we can model the true probabilities then we can mathematically calculate the optimal policy. Not all real world situations use dice, but these techniques work well even if we can only estimate the true probabilities.

You can play the game if you uncomment and run the line in the cell below. *Make sure to comment out this cell before submitting.*

In [ ]:
# Uncomment the line below to play the game.
# You *must* comment it out again before submitting.
# %run dice_game.py

## Value Iteration
The algorithm you should use to solve this game is called value iteration. It works by calculating the expected value for each action available from each state, and taking the maximum one. This value, for a given state and action, is calculated by taking a sum of the immediate reward and the expected value of the next state (which itself must be calculated as a probability-weighted sum of expected values for each possible next state). Since the values used in the second part of the sum are just estimates, the process must be repeated, updating each state, until convergence within a certain tolerance.

Here is some pseudocode for the value iteration algorithm. <br />

<img src="images/valueiteration.png" width=400 />

In the pseudocode above, $\mathcal{S}$ refers to the set of all non-terminal states and $\mathcal{S}^{+}$ refers to all states (including any terminal). Terminal states should always have a value of zero.

This pseudocode is based on the one from Sutton and Barto (page 83). The same technique in Russel and Norvig (page 652) has a more complicated termination criteron, but assumes the discount factor $\gamma < 1$. 

**For this assignment you should use $\gamma = 1$.** Because of the conditions of the game it is only possible to get a non-negative score by holding all three dice and transitioning to a terminal state. Since we know the game will terminate, we can use non-discounted rewards. 

You may wish to consult either or both textbook descriptions to better your understanding of the technique.

## Dice Game Class
A class called `DiceGame` is provided within `dice_game.py` for you to use in your solution. Here is a demonstration of its features.

When you create a DiceGame object, by default you will get the rules as stated above. For part two of the assignment you may wish to modify the game mechanics, and you can do this using the constructor, for example:
```python
game = DiceGame(dice=4, sides=3, bias=np.array([0.1, 0.1, 0.8]), penalty=2)
```
will create a game where you roll 4 dice, each with 3 sides, where each one is far more likely to roll a 3 than they are to roll a 1 or a 2, and furthermore the penalty for rerolling is now 2 points instead of 1. *Note: this does not necessarily result in an interesting game.*

Once created, the `DiceGame` object can be run in two different modes, *simulation* and *analysis*. For this assignment you will mostly use [*analysis* mode](#Analysis-Mode), but some understanding of simulation mode might be useful.

### Simulation Mode
The object provides the methods required to simulate playing the game. This might be useful if you want to test your policy, or you just want to try playing the game yourself, as we did in the cell earlier. The current dice values are found by calling `get_dice_state()`, they will always be listed in ascending order. 

In [ ]:
from dice_game import DiceGame
import numpy as np

# setting a seed for the random number generator makes testing easier!
np.random.seed(111)

game = DiceGame()
game.get_dice_state()

To roll the dice, you call the `roll` method which takes one parameter: a tuple representing which dice you want to hold, numbered from zero. We rolled (2, 3, 4). Suppose we want to hold the 2, we would pass the tuple `(0,)` into the `roll` method (note we need to include the comma so that Python knows this is a tuple).

The `roll` method returns a tuple containing: the reward for this action, the new state, and whether the game is over.

In [ ]:
reward, new_state, game_over = game.roll((0,))
print(reward)
print(new_state)
print(game_over)

Now suppose we are happy and wish to stick to get our final score. We can call the `roll` method and supply a tuple containing all three dice.

In [ ]:
reward, new_state, game_over = game.roll((0, 1, 2))
print(reward)
print(new_state)
print(game_over)

Remember that the return value is just the reward for the action, in this case 15. To get our final score we can inspect `game.score`. We rerolled once, so expect to get a score of 14.

In [ ]:
print(game.score)

And to play again we can call `game.reset()` which returns the new starting dice.

In [ ]:
game.reset()

### Analysis Mode
In analysis mode, you are not playing the game, but asking the object for all possible outcomes of certain actions. This will be used in the value iteration algorithm.

First of all, it is useful to know that all the possible states and all the possible actions are stored inside the object:

In [ ]:
game = DiceGame()
print(f"The first 5 of {len(game.states)} possible dice rolls are: {game.states[0:5]}")
print(f"The possible actions on any given turn are: {game.actions}")

Finally, the most important method is `get_next_states(action, dice_state)`. This allows you to get all the possible resulting states for any given state and action.

Earlier we had the roll of `(2, 3, 4)` and decided to hold the 2. The game can calculate all possible outcomes for us, and crucially will also give us the probability of each state occurring.

In [ ]:
states, game_over, reward, probabilities = game.get_next_states((0,), (2, 3, 4))
for state, probability in zip(states, probabilities):
    print(f"Would get roll of {state} with probability {probability}")

The method also works consistently when all dice are held, reporting that this action would cause the game to be over and giving the reward. Note that the list of states returned is empty, since we have no specific way to denote that a state is terminal. This is slightly inconsistent with simulation mode which shows the final dice after flipping duplicates (which is mainly for user friendliness).

In [ ]:
states, game_over, reward, probabilities = game.get_next_states((0, 1, 2), (2, 2, 5))
print(game_over)
print(reward)

## Part One
Write your code for value iteration in the cell below. This part is worth 80% of the assignment.

**Your solution must contain a method called `solve(game, theta)` which returns a dictionary mapping from each state to the best action.**
* You must use the values of the two parameters `game` and `theta` in your solution:
 * `game` is a DiceGame object – you should use this to query the game as described [above](#Analysis-Mode). If you write your solution correctly, you should be able to change the game rules by changing the constructor (e.g. changing the number of dice), and your value iteration algorithm should still work.
 * `theta` is the tolerance value $\theta$, after which you will assume convergence, as shown in the pseudocode [above](#Value-Iteration). You should experiment to determine what value of $\theta$ you think is best.

In the cell after next there are a few sample tests. You should run these tests to ensure your code is working correctly. ***Submissions which fail to meet these basic tests may receive zero marks.***

In [8]:
import numpy as np
from dice_game import DiceGame


from abc import ABC, abstractmethod


class DiceGameAgent(ABC):
    def __init__(self, game):
        self.game = game
    
    @abstractmethod
    def play(self, state):
        pass

class DiceAgent(DiceGameAgent):
    def __init__(self, game):
        
        super().__init__(game)
        self.num_actions = len(self.game.actions)
        self.Values = self.solve()

    
    def expected_rewards(self, Values, state):
        
        Actions = {a:0 for a in self.game.actions}

        # For each possible action
        for a in Actions.keys():
            
            # Get the possible next_states, game_over, reward and probabilities of the next state occurring
            next_states, game_over, reward, probabilities = self.game.get_next_states(a, state)
            
            # For each next_state and associated probability of that action & given state
            for next_state, probability in zip(next_states, probabilities): 
                # If there is a next_state
                if not game_over:
                    # Calculate Bellman Update
                    Actions[a] += probability * (reward+Values[next_state][0])
                else:
                    # next_state = None
                    Actions[a] += probability * reward
        return Actions


    def solve(self, theta = 0.1):
        
        
        # Initialise dictionary with each state as keys and values [expected reward, optimal policy]
        Values = {i: [0, None] for i in self.game.states}
        
        # Loop until delta < self.theta
        while True:
            delta = 0
    
            # For each state
            for s in self.game.states:
                # Get the list of reward values pertaining to each action given that state
                A = self.expected_rewards(Values, s)
                
                # Take the highest expected reward value 
                best_action_value = max(A.values())
                
                # Update delta with either delta or the absolute difference between the 
                # new maximum value for that state and the previous maximum value for that state
                delta = max(delta, np.abs(best_action_value - Values[s][0]))
                
                # Update the maximum value and optimal policy for that state   
                Values[s] = [best_action_value,max(A, key = A.get)]
                
            # If the change is less than self.theta, end the loop
            if delta < theta:
                break
        return Values

        
    def play(self, state):
        return self.Values[state][1]

    
game = DiceGame()
agent = DiceAgent(game)
policy  = agent.solve()

In [11]:
# There should be exactly 56 states
assert(len(policy.items()) == 56)

# The optimal move when you roll three 1s is to stick
assert(policy[(1, 1, 1)][1] == (0, 1, 2))

# The optimal move when you roll three 4s is to reroll all 3
assert(policy[(4, 4, 4)][1] == ())

print("All three tests pass.")

All three tests pass.


In [10]:
# This cell contains three sample tests

# Get your policy, a dictionary mapping from state to (best) action
policy = solve(game=DiceGame())

# There should be exactly 56 states
assert(len(policy.keys()) == 56)

# The optimal move when you roll three 1s is to stick
assert(policy[(1, 1, 1)] == (0, 1, 2))

# The optimal move when you roll three 4s is to reroll all 3
assert(policy[(4, 4, 4)] == ())

print("All three tests pass.")

NameError: name 'solve' is not defined

## Part Two
For this part, you need to write a short discussion and analysis of your implementation in the cell below. This part is worth 20% of the assignment. 
* Short means approximately 300 words. There are no hard limits but overly long answers may be penalised.
* You may wish to discuss topics such as:
 * Any choices you made during implementation
 * Any parameter values you used and the effect of choosing them differently
 * What impact there is on the results of your value iteration algorithm if run on a modified version of the game with different mechanics (as detailed [above](#Dice-Game-Class))
 * How efficient your code is and what improvements could be made

This agent chooses the most rewarded action to solve a game of dice.  The value iteration algorithm is applied to choose the best action, and the one-step look ahead heuristic to obtain the best score. A dictionary (“Values”) that contains every conceivable state as keys and a list of the expected utility/reward and the best policy for each state as values is computed during agent initialization. By iterating over each action accessible for each state, calculating every potential next state attainable from that action, as well as the related rewards gained and probability of reaching that state, the expected rewards are determined using the Bellman Update equation. As the one-step look ahead relies on next state values stored within “Values”, which are initialized to zero. The entire operation is carried out within a while-loop. The loop finally converges and ends when each state's predicted reward is updated or re-calibrated. Up till the change in predicted values is smaller than a set threshold, theta, the value iteration continues. The game starts whenever “Values” converges. By exploring the provided situation to find the best policy for each throw of the dice, the appropriate course of action is chosen. The agent will only ever carry out the stated action, hence it is important to note that the actions are deterministic in their nature. This makes it easier to calculate the Bellman update. I tested several values for the theta parameter, to determine the optimal value. Theta= 0.1 results in the quickest time per game and the highest score. PS I had to modify the cell containing the test code because otherwise i could not test my code. The agent satisfies the test that you provide.

This is the end of the assignment. The cells below contain our test code for part one, *you must not delete or modify them.*

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 